## 生成选通先验图

In [1]:
import tqdm
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
import torch


def readlines(filename):
    """Read all the lines in a text file and return as a list
    """
    with open(filename, 'r') as f:
        lines = f.read().splitlines()
    return lines


def read_img(img_path,
             num_bits=10,
             crop_height=420, crop_width=980, dataset='g2d'):
    gated_imgs = []
    normalizer = 2 ** num_bits - 1.

    for gate_id in range(3):
        path = img_path.format(gate_id)
        assert os.path.exists(path), "No such file : %s" % path
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        img = img[((img.shape[0] - crop_height) // 2):((img.shape[0] + crop_height) // 2),
              ((img.shape[1] - crop_width) // 2):((img.shape[1] + crop_width) // 2)]
        img = img.copy()
        img[img > 2 ** 10 - 1] = normalizer
        img = np.float32(img / normalizer)
        gated_imgs.append(np.expand_dims(img, axis=2))
    img = np.concatenate(gated_imgs, axis=2)
    return img


#name = ['train','val','test']
name = ['train']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_path = r'C:\Users\edric\PycharmProjects\newG2D\data\real\splits'
fpath = os.path.join(data_path, "real_{}_night.txt")
base_dir = r'C:\Users\edric\PycharmProjects\newG2D\data\real'
ppath = r'C:\Users\edric\PycharmProjects\newG2D\data\real\gated3_10bit_edric'

for n in range(len(name)):
    train_filenames = readlines(fpath.format(name[n]))
    img_id = {}
    for i in tqdm.tqdm(range(0, len(train_filenames))):
        # print("{}/{}".format(i, len(train_filenames)))
        img_id[i] = train_filenames[i].split('\n')
        id = img_id[i][0]
        gate_dir = os.path.join(base_dir, 'gated{}_10bit', '{}.png'.format(id))
        in_img = read_img(gate_dir)
        in_img = torch.tensor(in_img).unsqueeze(0).to(device=device) * 255
        img_min, _ = torch.min(in_img, dim=3)
        in_img[:, :, :, 0] = in_img[:, :, :, 0] - img_min
        in_img[:, :, :, 1] = in_img[:, :, :, 1] - img_min
        in_img[:, :, :, 2] = in_img[:, :, :, 2] - img_min
        img_max, max_index = torch.max(in_img, dim=3)
        s1 = img_max < 30
        s2 = img_max > 220
        max_index[s1] = -1
        max_index[s2] = -1
        max_index[max_index == 0] = 30
        max_index[max_index == 1] = 100
        max_index[max_index == 2] = 180
        max_index[max_index == -1] = 0

        cv2.imwrite(os.path.join(ppath, '{}.png'.format(id)), np.float32(max_index[0, :, :].cpu()))
        
        # plt.imshow(np.float32(max_index[0, :, :].cpu()), cmap='gray')  
        # # plt.colorbar()
        # plt.title("Gated Depth Prior)
        # plt.axis('off')
        # plt.show()
        # break




100%|██████████| 4014/4014 [03:35<00:00, 18.61it/s]


In [1]:

# 创建参数对象
from src.optionsClass import TrainingOptions

# Create an instance of TrainingOptions
opt = TrainingOptions()

# Set the values as per the shell commands
opt.data_dir = "./data"
opt.log_dir = "./logs"
opt.coeff_fpath = "chebychev"
opt.depth_flat_world_fpath = "depth_flat_world"
opt.model_name = "multinetwork"
opt.model_type = "multinetwork"
opt.exp_name = "multinetwork"
opt.models_to_load = ["depth", "ambient", "encoder", "albedo", "pose", "pose_encoder"]
opt.load_weights_folder = "./models/initialization"
opt.exp_num = 0
opt.height = 512
opt.width = 1024
opt.num_bits = 10
opt.scales = [0]
opt.frame_ids = [0, -1, 1]
opt.pose_model_type = "separate_resnet"
opt.num_layers = 18
opt.weights_init = "pretrained"
opt.pose_model_input = "pairs"
opt.min_depth = 0.1
opt.max_depth = 100.0
opt.dataset = "gated"
opt.split = "gated2gated"
opt.batch_size = 4
opt.num_workers = 4
opt.learning_rate = 2e-4
opt.num_epochs = 20
opt.scheduler_step_size = 15
opt.disparity_smoothness = 0.001
opt.log_frequency = 200
opt.save_frequency = 1
opt.cycle_weight = 0.05
opt.depth_normalizer = 70.0
opt.passive_weight = 0.01
opt.cycle_loss = True
opt.temporal_loss = True
opt.sim_gated = True
opt.v1_multiscale = True
opt.infty_hole_mask = True
opt.snr_mask = True
opt.intensity_mask = True
opt.passive_supervision = True

import torch

cudaIf = torch.cuda.is_available()
device = torch.device("cuda" if cudaIf else "cpu")
print("device=", device)


device= cuda


In [2]:
from src import networks

# 创建一个torch张量
x = torch.randn(1, 3, 512, 1024).to(device)

parameters_to_train = []
depth = networks.PackNetSlim01(
    dropout=opt.dropout,
    version="{}{}".format(1, opt.feat_stack)
)
depth.to(device)
if not opt.train_depth_normalizer:
    parameters_to_train += list(depth.parameters())

depth_out = depth(x)
print('depth.parameters: ', len(list(depth.parameters())))


PackNetSlim01 -- depth
torch.Size([1, 3, 512, 1024])--x_in.shape
torch.Size([1, 32, 512, 1024])--x.shape
torch.Size([1, 32, 512, 1024])--x1.shape
torch.Size([1, 32, 256, 512])--x1p.shape
torch.Size([1, 64, 256, 512])--x2.shape
torch.Size([1, 64, 128, 256])--x2p.shape
torch.Size([1, 128, 128, 256])--x3.shape
torch.Size([1, 128, 64, 128])--x3p.shape
torch.Size([1, 256, 64, 128])--x4.shape
torch.Size([1, 256, 32, 64])--x4p.shape
torch.Size([1, 512, 32, 64])--x5.shape
torch.Size([1, 512, 16, 32])--x5p.shape
torch.Size([1, 512, 32, 64])--unpack5.shape
torch.Size([1, 768, 32, 64])--concat5.shape
torch.Size([1, 512, 32, 64])--iconv5.shape
torch.Size([1, 256, 64, 128])--unpack4.shape
torch.Size([1, 384, 64, 128])--concat4.shape
torch.Size([1, 256, 64, 128])--iconv4.shape
torch.Size([1, 1, 64, 128])--disp4.shape
torch.Size([1, 1, 128, 256])--udisp4.shape
torch.Size([1, 128, 128, 256])--unpack3.shape
torch.Size([1, 193, 128, 256])--concat3.shape
torch.Size([1, 128, 128, 256])--iconv3.shape
torc

In [3]:
from src.trainer import Trainer

trainer = Trainer(opt)
trainer.train()

AssertionError: Cannot find folder ./models/initialization